# Huggingface 🤗 NLP Course

Neste curso serão abordados tópicos sobre Natural Language Processing (NLP) usando as bibliotecas do ecossistem a do [hugging face](https://huggingface.co/), como 🤗 Transformers, 🤗 Datasets, 🤗 Tokenizers, and 🤗 Accelerate — assim como Hugging Face Hub.

## Visão Geral
- Os capítulos 1 a 4 fornecem uma introdução aos principais conceitos da biblioteca 🤗 Transformers. Ao final desta parte do curso, você estará familiarizado com o funcionamento dos modelos do Transformer e saberá como usar um modelo do Hugging Face Hub, ajustá-lo em um conjunto de dados e compartilhar seus resultados no Hub!
- Os capítulos 5 a 8 ensinam o básico de 🤗 Conjuntos de dados e 🤗 Tokenizers antes de mergulhar nas tarefas clássicas de PNL. Ao final desta parte, você será capaz de resolver sozinho os problemas mais comuns da PNL.
- Os capítulos 9 a 12 vão além da PNL e exploram como os modelos Transformer podem ser usados para lidar com tarefas de processamento de fala e visão computacional. Ao longo do caminho, você aprenderá como criar e compartilhar demonstrações de seus modelos e otimizá-los para ambientes de produção. Ao final desta parte, você estará pronto para aplicar 🤗 Transformers a (quase) qualquer problema de aprendizado de máquina!

# 1. Modelos Transformers

## O que é NLP?
NLP é um campo da linguística e da Aprendizagem de Máquina (ML) focada em entender tudo relacionado a linguagem humana. O objetivo das tarefas de NLP não é apenas entender palavras soltas individualmente, mas ser capaz de entender o contexto dessas palavras.

A seguir uma lista de tarefas comuns de NLP, com alguns exemplos:

- Classificação de sentenças completas: Capturar o sentimento de uma revisão, detectar se um email é spam, determinar se a sentença é gramaticalmente correta ou onde duas sentenças são logicamente relacionadas ou não
- Classificação de cada palavra em uma sentença: Identificar os componentes gramaticais de uma sentença (substantivo, verbo, adjetivo), ou as entidades nomeadas (pessoa, local, organização)
- Geração de conteúdo textual: Completar um trecho com autogeração textual, preenchendo as lacunas em um texto com palavras mascaradas
Extrair uma resposta de um texto: Dada uma pergunta e um contexto, extrair a resposta baseada na informação passada no contexto
- Gerar uma nova sentença a partir de uma entrada de texto: Traduzir um texto para outro idioma, resumi-lo

NLP não se limita ao texto escrito. Também engloba desafios complexos nos campos de reconhecimento de discurso e visão computacional, tal como a geração de transcrição de uma amostra de áudio ou a descrição de uma imagem.


## Por que isso é desafiador?
Os computadores não processam a informação da mesma forma que os seres humanos. Por exemplo, quando nós lemos a sentença “Estou com fome”, nós podemos facilmente entender seu significado. Similarmente, dada duas sentenças como “Estou com fome” e “Estou triste”, nós somos capazes de facilmente determinar quão similares elas são. Para modelos de Aprendizagem de Máquina (ML), tarefas como essas são mais difíceis. O texto precisa ser processado de um modo que possibilite o modelo aprender por ele. E porque a linguagem é complexa, nós precisamos pensar cuidadosamente sobre como esse processamento tem que ser feito. Tem se feito muita pesquisa sobre como representar um texto e nós iremos observar alguns desses métodos no próximo capítulo.



## Transformers, o que eles podem fazer?

O objeto mais básico na biblioteca 🤗 Transformers é a função pipeline() . Ela conecta o modelo com seus passos necessários de pré e pós-processamento, permitindo-nos a diretamente inserir qualquer texto e obter uma resposta inteligível.


Há três principais passos envolvidos quando você passa algum texto para um pipeline:

O texto é pré-processado para um formato que o modelo consiga entender.
As entradas (inputs) pré-processados são passadas para o modelo.
As predições do modelo são pós-processadas, para que então você consiga atribuir sentido a elas.
Alguns dos pipelines disponíveis atualmente, são:

- feature-extraction (pega a representação vetorial do texto)
- fill-mask (preenchimento de máscara)
- ner (reconhecimento de entidades nomeadas)
- question-answering (responder perguntas)
- sentiment-analysis (análise de sentimentos)
- summarization (sumarização)
- text-generation (geração de texto)
- translation (tradução)
- zero-shot-classification (classificação “zero-shot”)

In [5]:
# !pip install datasets evaluate transformers[sentencepiece]

In [6]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598048329353333}]

In [7]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

### Classificação Zero-shot
Nós começaremos abordando uma tarefa mais desafiadora da qual nós precisamos classificar texto que não tenham sido rotulados. Esse é um cenário comum nos projetos reais porque anotar texto geralmente consome bastante do nosso tempo e requer expertise no domínio. Para esse caso, o pipeline zero-shot-classification é muito poderoso: permite você especificar quais rótulos usar para a classificação, desse modo você não precisa “confiar” nos rótulos dos modelos pré-treinados. Você já viu como um modelo pode classificar uma sentença como positiva ou negativa usando esses dois rótulos - mas também pode ser classificado usando qualquer outro conjunto de rótulos que você quiser. Esse pipeline é chamado de zero-shot porque você não precisa fazer o ajuste fino do modelo nos dados que você o utiliza.

In [8]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445989489555359, 0.11197412759065628, 0.04342695698142052]}

### Geração de Texto
Agora vamos ver como usar um pipeline para gerar uma porção de texto. A principal ideia aqui é que você coloque um pedaço de texto e o modelo irá autocompletá-lo ao gerar o texto restante. Isso é similar ao recurso de predição textual que é encontrado em inúmeros celulares. A geração de texto envolve aleatoriedade, então é normal se você não obter o mesmo resultado obtido mostrado abaixo. Você pode controlar quão diferentes sequências são geradas com o argumento num_return_sequences e o tamanho total da saída de texto (output) com o argumento max_length.

In [9]:
from transformers import pipeline

generator = pipeline("text-generation")
generator(
    "In this course, we will teach you how to"
)  # nesse curso, nós te mostraremos como você

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to integrate Android into Android Studio, which will help achieve high performance and maintainability in your project. With this course (which is available as an MP3 download), we will work together with you and your'}]

In [10]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to get involved in public and private businesses and the benefits of these new jobs and what you can do to'},
 {'generated_text': 'In this course, we will teach you how to use JavaScript. This content will be hosted in the browser and on a mobile device.\n\n\n'}]

### Preenchimento de máscara (*Mask filling*)
O próximo pipeline que você irá testar é o fill-mask. A ideia dessa tarefa é preencher os espaços em branco com um texto dado. O argumento top_k controla quantas possibilidades você quer que sejam geradas. Note que aqui o modelo preenche com uma palavra <máscara> especial, que é frequentemente referida como mask token. Outros modelos de preenchimento de máscara podem ter diferentes mask tokens, então é sempre bom verificar uma palavra máscara apropriada quando explorar outros modelos. Um modo de checar isso é olhando para a palavra máscara usada no widget.

In [11]:
from transformers import pipeline

unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.19619806110858917,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical models.'},
 {'score': 0.04052723944187164,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'This course will teach you all about computational models.'}]

### Reconhecimento de entidades nomeadas
Reconhecimento de Entidades Nomeadas (NER) é uma tarefa onde o modelo tem de achar quais partes do texto correspondem a entidades como pessoas, locais, organizações. Vamos olhar em um exemplo. Aqui o modelo corretamente identificou que Sylvain é uma pessoa (PER), Hugging Face é uma organização (ORG), e Brooklyn é um local (LOC).

Nós passamos a opção grouped_entities=True na criação da função do pipelina para dize-lo para reagrupar juntos as partes da sentença que correspondem à mesma entidade: aqui o modelo agrupou corretamente “Hugging” e “Face” como única organização, ainda que o mesmo nome consista em múltiplas palavras. Na verdade, como veremos no próximo capítulo, o pré-processamento até mesmo divide algumas palavras em partes menores. Por exemplo, Sylvain é dividido em 4 pedaços: S, ##yl, ##va, e ##in. No passo de pós-processamento, o pipeline satisfatoriamente reagrupa esses pedaços.

In [12]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'PER',
  'score': 0.9981694,
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': 0.9796019,
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': 0.9932106,
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]

### question-answering
O pipeline question-answering responde perguntas usando informações dado um contexto. Note que o pipeline funciona através da extração da informação dado um contexto; não gera uma resposta.

In [13]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.6949767470359802, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}

### Sumarização
Sumarização é uma tarefa de reduzir um texto em um texto menor enquanto pega toda (ou boa parte) dos aspectos importantes do texto referenciado. Como a geração de texto, você pode especificar o tamanho máximo max_length ou mínimo min_length para o resultado

In [14]:
from transformers import pipeline

summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': ' The number of engineering graduates in the United States has declined in recent years . China and India graduate six and eight times as many traditional engineers as the U.S. does . Rapidly developing economies such as China continue to encourage and advance the teaching of engineering . There are declining offerings in engineering subjects dealing with infrastructure, infrastructure, the environment, and related issues .'}]

### Tradução
Para tradução, você pode usar o modelo default se você der um par de idiomas no nome da tarefa (tal como "translation_en_to_fr", para traduzir inglês para francês), mas a maneira mais fácil é pegar o moddelo que você quiser e usa-lo no Model Hub. Aqui nós iremos tentar traduzir do Francês para o Inglês. Como a geração de texto e a sumarização, você pode especificar o tamanho máximo max_length e mínimo min_length para o resultado.

In [15]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'This course is produced by Hugging Face.'}]

## Como transformers funciona ?

Em linhas gerais, eles podem ser agrupados em três categorias:

- GPT-like (também chamados de modelos Transformers auto-regressivos)
- BERT-like (também chamados de modelos Transformers auto-codificadores)
- BART/T5-like (também chamados de modelos Transformers sequence-to-sequence)

### Transformers são modelos de linguagem
Todos os modelos de Transformer mencionados acima (GPT, BERT, BART, T5, etc.) foram treinados como ```modelos de linguagem (langauge models)```. Isso significa que eles foram treinados em grandes quantidades de texto bruto de forma auto-supervisionada. O aprendizado autossupervisionado é um tipo de treinamento no qual o objetivo é calculado automaticamente a partir das entradas do modelo. Isso significa que os humanos não são necessários para rotular os dados!

Este tipo de modelo desenvolve uma compreensão estatística da linguagem em que foi treinado, mas não é muito útil para tarefas práticas específicas. Por causa disso, o modelo geral pré-treinado passa por um processo chamado ```transfer learning```. Durante esse processo, o modelo é ajustado de maneira supervisionada - ou seja, usando rótulos anotados por humanos - em uma determinada tarefa.

Um exemplo de tarefa é prever a próxima palavra em uma frase depois de ler as n palavras anteriores. Isso é chamado de ```modelagem de linguagem causal (causal language modeling)``` porque a saída depende das entradas passadas e presentes, mas não das futuras.

### Transformers são modelos grandes
Além de alguns outliers (como o DistilBERT), a estratégia geral para obter melhor desempenho é aumentar os tamanhos dos modelos, bem como a quantidade de dados em que são pré-treinados.

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/model_parameters.png">

Pré-treinamento é o ato de treinar um modelo do zero: os pesos são inicializados aleatoriamente e o treinamento começa sem nenhum conhecimento prévio.

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/pretraining.svg">

Esse pré-treinamento geralmente é feito em grandes quantidades de dados. Portanto, requer um corpus de dados muito grande e o treinamento pode levar várias semanas.

```Ajuste fino (fine tuning)```, por outro lado, é o treinamento feito **após** um modelo ter sido pré-treinado. Para realizar o ajuste fino, primeiro você adquire um modelo de linguagem pré-treinado e, em seguida, realiza treinamento adicional com um conjunto de dados específico para sua tarefa. Espere - por que não simplesmente treinar diretamente para a tarefa final? Existem algumas razões:

- O modelo pré-treinado já foi treinado em um conjunto de dados que possui algumas semelhanças com o conjunto de dados de ajuste fino. O processo de ajuste fino é, portanto, capaz de aproveitar o conhecimento adquirido pelo modelo inicial durante o pré-treinamento (por exemplo, com problemas de NLP, o modelo pré-treinado terá algum tipo de compreensão estatística da linguagem que você está usando para sua tarefa).
- Como o modelo pré-treinado já foi treinado com muitos dados, o ajuste fino requer muito menos dados para obter resultados decentes.
Pela mesma razão, a quantidade de tempo e recursos necessários para obter bons resultados são muito menores.

Por exemplo, pode-se aproveitar um modelo pré-treinado treinado no idioma inglês e, em seguida, ajustá-lo em um corpus arXiv, resultando em um modelo baseado em ciência/pesquisa. O ajuste fino exigirá apenas uma quantidade limitada de dados: o conhecimento que o modelo pré-treinado adquiriu é “transferido”, daí o termo ```transfer learning```.

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/finetuning.svg">

O ajuste fino de um modelo de linguagem é mais barato do que o pré-treinamento, tanto em tempo quanto em dinheiro. Portanto, acarreta menores custos de tempo, dados, financeiros e ambientais. Também é mais rápido e fácil iterar diferentes esquemas de ajuste fino, pois o treinamento é menos restritivo do que um pré-treinamento completo.

Este processo também alcançará melhores resultados do que treinar do zero (a menos que você tenha muitos dados), e é por isso que você deve sempre tentar aproveitar um modelo pré-treinado – um que seja o mais próximo possível da tarefa que você tem em mãos – e ajustar isto.

### Arquitetura geral

O modelo transformer é composto principalmente por dois blocos:

- Encoder/Codificador (esquerda): O codificador recebe uma entrada e constrói uma representação dela (seus recursos). Isso significa que o modelo é otimizado para adquirir entendimento a partir da entrada.
- Decoder/Decodificador (direita): O decodificador usa a representação (recursos) do codificador junto com outras entradas para gerar uma sequência alvo. Isso significa que o modelo está otimizado para gerar resultados.

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/transformers_blocks.svg">


Cada uma dessas partes pode ser usada de forma independente, dependendo da tarefa:

- Encoder-only models: bons para tarefas que exigem compreensão da entrada, como **classificação de frases** e **reconhecimento de entidade nomeada**.
- Decoder-only models: bons para tarefas generativas, como **geração de texto**.
- Encoder-decoder models ou sequence-to-sequence models: bons para tarefas generativas que requerem uma entrada, como **tradução** ou **resumo**.

### Camadas de Atenção

Uma característica importante dos modelos Transformer é que eles são construídos com camadas especiais chamadas **camadas de atenção**. Na verdade, o título do artigo que apresenta a arquitetura do Transformer era _“Attention is all you need”. Esta camada dirá ao modelo para prestar atenção específica a certas palavras da frase que você passou (e mais ou menos ignorar as outras) ao lidar com a representação de cada palavra.

Para contextualizar isso, considere a tarefa de traduzir um texto do inglês para o francês. Dada a entrada “Você gosta deste curso”, um modelo de tradução precisará também atender à palavra adjacente “Você” para obter a tradução adequada para a palavra “gosto”, porque em francês o verbo “gosto” é conjugado de forma diferente dependendo de o sujeito. O resto da frase, porém, não é útil para a tradução dessa palavra. Na mesma linha, ao traduzir “isto” o modelo também precisará prestar atenção à palavra “curso”, porque “isto” é traduzido de forma diferente dependendo se o substantivo associado é masculino ou feminino. Novamente, as outras palavras da frase não importarão para a tradução de “isto”. Com frases mais complexas (e regras gramaticais mais complexas), o modelo precisaria prestar atenção especial às palavras que possam aparecer mais distantes na frase para traduzir corretamente cada palavra.

O mesmo conceito se aplica a qualquer tarefa associada à linguagem natural: uma palavra por si só tem um significado, mas esse significado é profundamente afetado pelo contexto, que pode ser qualquer outra palavra (ou palavras) antes ou depois da palavra em estudo.

### A arquitetura original

A arquitetura Transformer foi originalmente projetada para tradução. Durante o treinamento, o **codificador recebe entradas (frases) em um determinado idioma**, enquanto o **decodificador recebe as mesmas frases no idioma de destino desejado**. No codificador, **as camadas de atenção podem usar todas as palavras em uma frase** (já que, como acabamos de ver, a tradução de uma determinada palavra pode ser dependente do que está depois e antes dela na frase). **O decodificador, no entanto, funciona sequencialmente e só pode prestar atenção nas palavras da frase que ele já traduziu** (portanto, apenas as palavras anteriores à palavra que está sendo gerada no momento). Por exemplo, quando previmos as três primeiras palavras do alvo traduzido, as entregamos ao decodificador que então usa todas as entradas do codificador para tentar prever a quarta palavra.

Para acelerar as coisas durante o treinamento (quando o modelo tem acesso às frases alvo), o decodificador é alimentado com todo o alvo, mas não é permitido usar palavras futuras (se teve acesso à palavra na posição 2 ao tentar prever a palavra na posição 2, o problema não seria muito difícil!). Por exemplo, ao tentar prever a quarta palavra, a camada de atenção só terá acesso às palavras nas posições 1 a 3.

A arquitetura original do Transformer ficou assim, com o codificador à esquerda e o decodificador à direita:

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter1/transformers.svg">

Observe que **a primeira camada de atenção em um bloco decodificador presta atenção a todas as entradas (passadas) do decodificador**, mas a **segunda camada de atenção usa a saída do codificador**. Ele pode, assim, acessar toda a frase de entrada para melhor prever a palavra atual. Isso é muito útil, pois diferentes idiomas podem ter regras gramaticais que colocam as palavras em ordens diferentes, ou algum contexto fornecido posteriormente na frase pode ser útil para determinar a melhor tradução de uma determinada palavra.

A _máscara de atenção_ (attention mask) também pode ser usada no codificador/decodificador para evitar que o modelo preste atenção a algumas palavras especiais - por exemplo, a palavra de preenchimento especial usada para fazer com que todas as entradas tenham o mesmo comprimento ao agrupar frases.

### Arquiteturas vs. checkpoints
- **Arquitetura**: Este é o esqueleto do modelo — a definição de cada camada e cada operação que acontece dentro do modelo.
- **Checkpoints**: Esses são os pesos que serão carregados em uma determinada arquitetura.
- **Modelos**: Este é um termo abrangente que não é tão preciso quanto “arquitetura” ou “checkpoint”: pode significar ambos. Este curso especificará arquitetura ou checkpoint quando for necessário reduzir a ambiguidade.
Por exemplo, BERT é uma arquitetura enquanto bert-base-cased, um conjunto de pesos treinados pela equipe do Google para a primeira versão do BERT, é um checkpoint. No entanto, pode-se dizer “o modelo BERT” e “o modelo bert-base-cased“.

## Encoder Models

Os modelos de **encoder** usam apenas o encoder de um modelo Transformer. Em cada estágio, as camadas de atenção podem acessar todas as palavras da frase inicial. Cada palavra, é representada numericamente através de um vetor contextualizado gerado pelo encoder. Essa a contextualização é criada através do mecanismo de **self-attention**. O vetor gerado é chamado de vetor de features ou tensor de features, e ele segue o tamanho (dimensão) do vetor é definida pela arquitetura do modelo, no caso do BERT base, o tamanho é de 768. 


Esses modelos geralmente são caracterizados como tendo atenção “bidirecional” e são frequentemente chamados de ```auto-encoding models```.

O pré-treinamento desses modelos geralmente gira em torno de corromper de alguma forma uma determinada frase (por exemplo, mascarando palavras aleatórias nela) e encarregando o modelo de encontrar ou reconstruir a frase inicial.

Os modelos de codificador são mais adequados para tarefas que exigem uma compreensão da sentença completa, como classificação de sentença, reconhecimento de entidade nomeada (e, mais geralmente, classificação de palavras) e resposta extrativa de perguntas.

Modelos encoder são bons em extrair informações contextuais. Esses modelos sãoa dequados para atividades como:
- Sequence Classification
- Question Answering
- Masked Langauge modeling
- NLU: Natural language understanding

Os representantes desta família de modelos incluem:
- ALBERT
- BERT
- DistilBERT
- ELECTRA
- RoBERTa

## Decoder Models

Os modelos de decodificador usam apenas o decodificador de um modelo Transformer. Em cada etapa, para uma determinada palavra, as camadas de atenção só podem acessar as palavras posicionadas antes dela na frase. Esses modelos geralmente são chamados de modelos ```auto-regressive models```. O pré-treinamento de modelos de decodificadores geralmente gira em torno de prever a próxima palavra na frase.

Os inputs passados no decoder também criam uma representação numérica das palavras, de forma contextualizada. Mas diferente do encoder, no encoder o self attention é chamado de **masked self attention**, e essas representações numéricas das palavras são criadas somente das palavras anteriores até a atual, **não são usadas todas as palavras**. Todas as palavras da direta são ocultas (mascaradas). Os modelos encoder **não** são bidirecionais, e sim, **unidirecional**. 

Esses modelos são mais adequados para tarefas que envolvem geração de texto. Como
- Causal tasks; geração de sequencias
- NLG: Natural language generation

Os representantes desta família de modelos incluem:
- CTRL
- GPT
- GPT-2
- Transformer XL

## Sequence-to-sequence models

Modelos **encoder-decoder** (também chamados de modelos **sequence-to-sequence**) usam ambas as partes da arquitetura Transformer. Em cada estágio, as camadas de atenção do encoder podem acessar todas as palavras da frase inicial, enquanto as camadas de atenção do decoder podem acessar apenas as palavras posicionadas antes de uma determinada palavra na entrada.

O pré-treinamento desses modelos pode ser feito usando os objetivos dos modelos de codificador ou decodificador, mas geralmente envolve algo um pouco mais complexo. Por exemplo, T5 é pré-treinado substituindo trechos aleatórios de texto (que podem conter várias palavras) por uma única palavra especial de máscara, e o objetivo é prever o texto que esta palavra de máscara substitui.

No encoder-decoder, além de receber os inputs inicialmente, ele também recebe os outputs do encoder, na segunda camada de atenção. Quando não é passado uma sequencia inicial, são atribuṕidos valores que indica o início de uma sequencia. O encoder recebe a sequencia como entrada, calcula uma previsão e gera a representação numérica. Depois, ele envia a sequencia encodada para o decoder, e ele usa essa entrada junto com sua entrada de sequencia do decoder e tenta decodificar a sequencia. Assim, atuando de forma autoregressiva prevendo a próxima palavra a partir das palavras atuais. 

Os modelos **seq2seq** são capazes de atuar em tarefas de:
- Tradução
- Summarization

Os representantes desta família de modelos incluem:
- BART
- mBART
- Marian
- T5

# Usando 🤗 Transformers

A biblioteca 🤗 Transformers foi criada para fornecer uma API única através do qual qualquer modelo Transformer possa ser carregado, treinado e salvo. As principais características da biblioteca são:

- Fácil de usar: Baixar, carregar e usar um modelo de processamento natural de linguagem (PNL) de última geração para inferência pode ser feito em apenas duas linhas de código
- Flexibilidade: Em sua essência, todos os modelos são uma simples classe PyTorch nn.Module ou TensorFlow tf.keras.Model e podem ser tratadas como qualquer outro modelo em seus respectivos frameworks de machine learning (ML).
- Simplicidade: Quase nenhuma abstração é feita em toda a biblioteca. O “Tudo em um arquivo” é um conceito principal: o “passe para frente” de um modelo é inteiramente definido em um único arquivo, de modo que o código em si seja compreensível e hackeável.

Esta última característica torna 🤗 Transformers bem diferente de outras bibliotecas ML que modelos e/ou configurações são compartilhados entre arquivos; em vez disso, cada modelo tem suas próprias camadas. Além de tornar os modelos mais acessíveis e compreensíveis, isto permite que você experimente facilmente um modelo sem afetar outros.

A função pipeline agrupa três processos: tokenizer, model e post processing
- Tokenizer: É responsável por criar a representação numérica do texto de entrada, e, depois retornar ao texto quando necessário. Recebe a entrada de texto e divide em palavras e palavras, adiciona tokens especiais (de início da sentença [CLS] e final [SEP]). Então, o tokenizer associa cada token ao seu id exclusivo do vocabulário do modelo pré-treinado.
- Model: Recebe as representações numéricas do texto de entrada, e retorna um output Logit* da tarefa executada. 
- Post processing: A etapa de pós processamento recebe a saída do model e aplica uma função de ativação, como softmax, para normalizar os valores de logit em probabilidade. O retorno do pós processamento é uma lista com dois valores (no caso da classificação), em que um representa a classe negativa e outro positiva. Para ver qual valor representa é qual, basta utilizar ```model.config.id2label```.

***Logit*** valores que não estão diretamente relacionados a probabilidades. Eles representam a saída da última camada da rede neural antes de aplicar uma função de ativação, como a função sigmoid ou softmax. Quanto maior o logit para uma classe, mais a rede neural está "pensando" que a imagem pertence a essa classe.

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg">

Exemplo

```python
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)
```

tem como resultado:

```python

[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]
 
```

Aprofundando cada etapa...

### Pré-processamento com o tokenizer

Como outras redes neurais, os modelos do Transformer não podem processar texto bruto diretamente, portanto, a primeira etapa do nosso pipeline é converter as entradas de texto em números que o modelo possa entender. Para isso utilizamos um tokenizer, que será responsável por:

- Dividir a entrada em palavras, subpalavras ou símbolos (como pontuação) que são chamados de tokens
Mapeando cada token para um número inteiro
- Adicionando entradas adicionais que podem ser úteis para o modelo
- Todo esse pré-processamento precisa ser feito exatamente da mesma maneira que quando o modelo foi pré-treinado, então primeiro precisamos baixar essas informações do **Model Hub**. Para fazer isso, usamos a classe ```AutoTokenizer``` e seu método ```from_pretrained()```. Aqui, instanciando o modelo no objeto ```checkpoint```, ele irá buscar automaticamente os dados associados ao tokenizer do modelo e armazená-los em cache (portanto, ele só será baixado na primeira vez que você executar o código abaixo).

Como o modelo padrão do pipeline de análise de sentimento é ```distilbert-base-uncased-finetuned-sst-2-english```, executamos o seguinte:

```python
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
```

Assim que tivermos o tokenizer, podemos passar diretamente nossas frases para ele e receberemos de volta um dicionário que está pronto para alimentar nosso modelo! A única coisa que falta fazer é converter a lista de identificações de entrada em tensores.

Você pode usar 🤗 Transformers sem ter que se preocupar com qual estrutura ML é usada como backend; pode ser PyTorch ou TensorFlow, ou Flax para alguns modelos. Entretanto, os Transformers só aceitam **tensores** como entrada. Os tensores são estruturas de dados fundamentais em matemática e no campo da aprendizagem de máquina, incluindo deep learning. Eles podem ser pensados como uma generalização de vetores e matrizes, que são estruturas unidimensionais e bidimensionais, respectivamente. Em termos simples, você pode imaginar tensores como recipientes que podem armazenar dados em diferentes formatos. Cada elemento dentro de um tensor é identificado por um conjunto de índices. Vou dar alguns exemplos para ilustrar:

1. **Tensor 0D (Escalar)**: Este é o tensor mais simples, representando um único número. Por exemplo, a temperatura de um dia (30 graus Celsius) pode ser armazenada em um tensor 0D.

2. **Tensor 1D (Vetor)**: Este é um tensor unidimensional, como uma lista de números. Um exemplo é a sequência de preços de ações ao longo de vários dias.

3. **Tensor 2D (Matriz)**: Aqui, você tem uma estrutura de tabela com linhas e colunas, como uma planilha. Uma imagem em tons de cinza pode ser representada como uma matriz, onde cada elemento representa a intensidade do pixel.

4. **Tensor 3D**: Este tipo de tensor é tridimensional e pode ser usado para representar dados como um conjunto de imagens coloridas, onde uma dimensão adiciona a informação de cor (por exemplo, vermelho, verde, azul).

5. **Tensor ND (Tensor Multidimensional)**: Tensores podem ter mais de três dimensões, tornando-os adequados para uma variedade de aplicações complexas. Por exemplo, vídeos podem ser representados como tensores 4D, onde as dimensões são largura, altura, tempo e canais de cor.

Os tensores são a base para armazenar e manipular dados em deep learning e matemática aplicada. Eles são usados para representar entradas, saídas e parâmetros em redes neurais, permitindo que modelos de aprendizado de máquina processem informações em várias dimensões e realizem operações matemáticas complexas. Portanto, tensores desempenham um papel crucial na análise de dados, visão computacional, processamento de linguagem natural e muitas outras aplicações.

Para especificar o tipo de tensores que queremos recuperar (PyTorch, TensorFlow ou NumPy), utilizamos o argumento ```return_tensors```:

```python
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)
```
O parametro ```padding```foi utilizado para criar matrizes do mesmo tamanho para frases com tamanhos diferentes, enquanto que o ```truncation``` foi usado pra limitar o tamanho da frase para o tamanho que o modelo suporta.

```python
{
    'input_ids': tensor([
        [  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172, 2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,     0,     0,     0,     0,     0,     0]
    ]), 
    'attention_mask': tensor([
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ])
}
```
Os resultados como tensores é um dicionário contendo duas chaves, ```input_ids``` e ```attention_mask```. O ```input_ids``` contém duas linhas de inteiros (uma para cada frase) que são os identificadores únicos dos tokens em cada frase. Jś os valores da chave ```attention_mask``` mostra onde foi aplicado o mecanismo de atenção, como as duas frases tem tamanhos diferentes, onde foi inserido o ```padding``` não foi aplicado o mecanismo de atenção.

### Etapa do modelo

Podemos baixar nosso modelo pré-treinado da mesma forma que fizemos com nosso tokenizer. 🤗 Transformers fornece uma classe AutoModel que também tem um método from_pretrained():

```python
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)
```

Neste trecho de código, fizemos o download do mesmo checkpoint que usamos anteriormente em nosso pipeline (já deveria estar em cache) e instanciamos um modelo com ele.

Esta arquitetura contém apenas o módulo base do Transformer: dadas algumas entradas, ele produz o que chamaremos de hidden states (estados ocultos), também conhecidos como features (características). Para cada modelo de entrada, recuperaremos um vetor de alta dimensionalidade representando a compreensão contextual dessa entrada pelo Transformer*.

A saída vetorial pelo módulo do Transformer é geralmente grande. Geralmente tem três dimensões:

- Tamanho do lote (Batch size): O número de sequências processadas de cada vez (2 em nosso exemplo).
- Tamanho da sequencia (Sequence length): O comprimento da representação numérica da sequência (16 em nosso exemplo).
- Tamanho oculto (Hidden size): A dimensão vetorial de cada modelo de entrada.

Diz-se que é “de alta dimensionalidade” por causa do último valor. O tamanho oculto pode ser muito grande (768 é comum para modelos menores, e em modelos maiores isto pode chegar a 3072 ou mais).

Podemos ver isso se alimentarmos os inputs que pré-processamos para nosso modelo:

```python
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])
```

Observe que as saídas dos 🤗 Transformer se comportam como ‘tuplas nomeadas’ (namedtuple) ou dicionários. Você pode acessar os elementos por atributos (como fizemos) ou por chave (outputs["last_hidden_state"]), ou mesmo por índice se você souber exatamente onde o que está procurando (outputs[0]).

As heads do modelo usam o vetor de alta dimensionalidade dos hidden states como entrada e os projetam em uma dimensão diferente. Eles são geralmente compostos de uma ou algumas camadas lineares:

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg">

A saída do Transformer é enviada diretamente para a head do modelo a ser processado.

Neste diagrama, o modelo é representado por sua camada de embeddings (vetores) e pelas camadas subsequentes. A camada de embeddings converte cada ID de entrada na entrada tokenizada em um vetor que representa o token associado. As camadas subsequentes manipulam esses vetores usando o mecanismo de atenção para produzir a representação final das sentenças.

Há muitas arquiteturas diferentes disponíveis no 🤗 Transformers, com cada uma projetada em torno de uma tarefa específica. Aqui está uma lista por algumas destas tarefas:

* Model (recuperar os hidden states)
* ForCausalLM
* ForMaskedLM
* ForMultipleChoice
* ForQuestionAnswering
* ForSequenceClassification
* ForTokenClassification
e outros 🤗

Para nosso exemplo, precisaremos de um modelo com uma head de classificação em sequencia (para poder classificar as sentenças como positivas ou negativas). Portanto, não utilizaremos a classe ```AutoModel```, mas sim, a classe ```AutoModelForSequenceClassification```:

```python
from transformers import AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
```

Agora se observarmos o tamanho dos nossos inputs, a dimensionalidade será muito menor: a head do modelo toma como entrada os vetores de alta dimensionalidade que vimos anteriormente, e os vetores de saída contendo dois valores (um por label):

```python
print(outputs.logits.shape)
torch.Size([2, 2])
```

Como temos apenas duas sentenças e duas labels, o resultado que obtemos de nosso modelo é de tamanho 2 x 2.

### Etapa de pós processamento
Os valores que obtemos como resultado de nosso modelo não fazem necessariamente sentido sozinhos. Vamos dar uma olhada:

```python
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward>)
```
Nosso modelo previu [-1.5607, 1.6123] para a primeira frase e [ 4.1692, -3.3464] para a segunda. Essas não são probabilidades, mas **logits**, a pontuação bruta e não normalizada produzida pela última camada do modelo. Para serem convertidos em probabilidades, eles precisam passar por uma camada **SoftMax** (todas saídas dos 🤗 Transformers produzem **logits**, já que a **função de loss (perda)** para treinamento geralmente fundirá a última função de ativação, como **SoftMax**, com a função de **loss real**, por exemplo a **cross entropy**), ou seja, a camada em que é aplicado o SoftMax é uma camada "pré-ativação":

```python
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward>)
```
Agora podemos ver que o modelo previu [0.0402, 0.9598] para a primeira frase e [0.9995, 0.0005] para a segunda. Estas são pontuações de probabilidade reconhecíveis.

Para obter as etiquetas correspondentes a cada posição, podemos inspecionar o atributo id2label da configuração do modelo (mais sobre isso na próxima seção):

```python
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}
```

Agora podemos concluir que o modelo previu o seguinte:
- A primeira frase: NEGATIVE: 0.0402, POSITIVE: 0.9598
- Segunda frase: NEGATIVE: 0.9995, POSITIVE: 0.0005

## Modelos

A classe ```AutoModel``` e todas as classes filhas são na verdade simples wrapper sobre a grande variedade de modelos disponíveis na biblioteca. É um wrapper inteligente, pois pode automaticamente “adivinhar” a arquitetura apropriada do modelo para seu checkpoint, e então instancia um modelo com esta arquitetura.

Entretanto, se você conhece o tipo de modelo que deseja usar, pode usar diretamente a classe que define sua arquitetura. Vamos dar uma olhada em como isto funciona com um modelo BERT.

### Criando um modelo
A primeira coisa que precisamos fazer para inicializar um modelo BERT é carregar um objeto de configuração:

```python
from transformers import BertConfig, BertModel

# Construindo a configuração
config = BertConfig()

# Construindo o modelo a partir da configuração
model = BertModel(config)
```

A configuração contém muitos atributos que são usados para construir o modelo:

```python
print(config)


BertConfig {
  [...]
  "hidden_size": 768,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  [...]
}
```

### Diferentes métodos de inicializar o modelo

A criação de um modelo a partir da configuração padrão o inicializa com valores aleatórios:

```python
from transformers import BertConfig, BertModel

config = BertConfig()
model = BertModel(config)

# O modelo é inicializado aleatoriamente!

```

O modelo pode ser utilizado neste estado, mas produzirá saídas errôneas; ele precisa ser treinado primeiro. Poderíamos treinar o modelo a partir do zero na tarefa em mãos, mas isto exigiria muito recurso, tempo e muitos dados. Para evitar esforços desnecessários e duplicados, normalmente é possível compartilhar e reutilizar modelos que já foram treinados.

Carregar um Transformer já treinado é simples - podemos fazer isso utilizando o método ```from_pretrained()```:


```python
from transformers import BertModel
model = BertModel.from_pretrained("bert-base-cased")
```

Como você viu anteriormente, poderíamos substituir o BertModel pela classe equivalente ao AutoModel. Faremos isto de agora em diante, pois isto produz um código generalista a partir de um checkpoint; se seu código funciona para checkpoint, ele deve funcionar perfeitamente com outro. Isto se aplica mesmo que a arquitetura seja diferente, desde que o checkpoint tenha sido treinado para uma tarefa semelhante (por exemplo, uma tarefa de análise de sentimento).

No exemplo de código acima não utilizamos BertConfig, e em vez disso carregamos um modelo pré-treinado através do identificador bert-base-cased. **Este é um checkpoint do modelo** que foi treinado pelos próprios autores do BERT; você pode encontrar mais detalhes sobre ele em seu [model card](https://huggingface.co/bert-base-cased).

Este modelo agora é inicializado com todos os pesos do checkpoint. Ele pode ser usado diretamente para inferência sobre as tarefas nas quais foi treinado, e também pode ser fine-tuned (aperfeiçoado) em uma nova tarefa. Treinando com pesos pré-treinados e não do zero, podemos rapidamente alcançar bons resultados.

Os pesos foram baixados e armazenados em cache (logo, para as futuras chamadas do método ```from_pretrained()``` não será realizado o download novamente) em sua respectiva pasta, que tem como padrão o path ```~/.cache/huggingface/transformers```. Você pode personalizar sua pasta de cache definindo a variável de ambiente HF_HOME.

O identificador usado para carregar o modelo pode ser o identificador de qualquer modelo no Model Hub, desde que seja compatível com a arquitetura BERT. A lista completa dos checkpoints BERT disponíveis podem ser encontrada [aqui].https://huggingface.co/models?filter=bert).

### Métodos para salvar/armazenar o modelo
Salvar um modelo é tão fácil quanto carregar um - utilizamos o método save_pretrained(), que é análogo ao método ```from_pretrained()```:

```python
model.save_pretrained("path_no_seu_computador")

# Isto salva dois arquivos em seu disco:

ls path_no_seu_computador
config.json pytorch_model.bin

```

Se você der uma olhada no arquivo config.json, você reconhecerá os atributos necessários para construir a arquitetura modelo. Este arquivo também contém alguns metadados, como a origem do checkpoint e a versão 🤗 Transformers que você estava usando quando salvou o checkpoint pela última vez.

O arquivo pytorch_model.bin é conhecido como o **dicionário de estado**; ele contém todos os pesos do seu modelo. Os dois arquivos andam de mãos dadas; **a configuração é necessária para conhecer a arquitetura de seu modelo**, enquanto **os pesos do modelo são os parâmetros de seu modelo**.

### Usando um modelo de Transformer para inferência
Os Tokenizers podem se encarregar de lançar as entradas nos tensores da estrutura apropriada, mas para ajudá-lo a entender o que está acontecendo, vamos dar uma rápida olhada no que deve ser feito antes de enviar as entradas para o modelo.

Digamos que temos um par de sequências

```python
sequences = ["Hello!", "Cool.", "Nice!"]

```

O tokenizer os converte em índices de vocabulário que são normalmente chamados de IDs de entrada. Cada sequência é agora uma lista de números! A saída resultante é:


```python
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

```

Esta é uma lista de sequências codificadas: uma lista de listas. Os tensores só aceitam shapes (tamanhos) retangulares (pense em matrizes). Esta “matriz” já é de forma retangular, portanto, convertê-la em um tensor é fácil:

```python
import torch
model_inputs = torch.tensor(encoded_sequences)
```

### Usando os tensores como entradas para o modelo

Fazer uso dos tensores com o modelo é extremamente simples - chamamos apenas o modelo com os inputs:

```python
output = model(model_inputs)
```

Embora o modelo aceite muitos argumentos diferentes, apenas os IDs de entrada são necessários. 

## Tokenizers

Os Tokenizers cuidam da primeira e da última etapa do processamento, cuidando da conversão de texto para entradas numéricas para a rede neural, e da conversão de volta ao texto quando for necessário. Eles são um dos componentes centrais do pipeline da PNL. Eles têm um propósito: traduzir texto em dados que podem ser processados pelo modelo. Os modelos só podem processar números, portanto os tokenizers precisam converter nossas entradas de texto em dados numéricos. O objetivo dos tokenizers é encontrar a representação mais significativa - ou seja, a que faz mais sentido para o modelo - e, se possível, a menor representação.





## Tratando Sequencias Múltiplas

## Agrupando Tudo